First of all, import all needed packages, particularly: **torch** and **torchvision**.

In [16]:
# Original Source: https://github.com/pytorch/examples/blob/master/mnist/main.py
# Code from: https://github.com/jiapei100/PyTorchZeroToAll/blob/master/10_1_cnn_mnist.py

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

Loading MNIST dataset. You'll see 4 datasets are downloaded during the running process.
* Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


In [17]:

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Afterwards, define CNN


In [18]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


Define training function

In [19]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


Define testing function

In [20]:

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train and test

In [21]:

for epoch in range(1, 10):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313586
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.281689
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297252
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.280714
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.270877
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.235533
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.230326
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.191298
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.165507
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.094295
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.977994
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.795815
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.640058
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.353927
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.186284
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.911913
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.701897
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.708441
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.927225
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.762540
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.101452
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.135212
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.167396
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.062063
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.125964
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.113294
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.230424
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.072031
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.166632
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.082398
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.133930
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.168134
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.123921
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.108696
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.099091
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.056073
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.074217
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.147233
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.087475
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.241883


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.189845
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.032497
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.034114
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.106197
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.055528
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.112526
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.096194
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.093490
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.100564
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.064140
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.132770
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.084953
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.084134
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.056889
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.053159
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.021863
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.150316
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.049434
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.096791
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.096946


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.067640
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.059775
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.032036
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.040946
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.024952
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.029708
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.052105
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.048344
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.034579
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.086917
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.037704
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.105882
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.318395
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.019278
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.027753
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.165485
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.099161
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.073175
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.025992
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.151996


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.042569
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.024059

Test set: Average loss: 0.0529, Accuracy: 9851/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.108963
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.103631
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.251000
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.014999
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.050609
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.040779
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.032287
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.048902
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.133694
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.068405
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.039177
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.072477
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.084655
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.019002
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.014680
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.030456
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.037332
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.060613
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.057037
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.140744
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.143096
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.015258
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.011869
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.261477
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.127271
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.078347
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.024347
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.004951
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.034131
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.022360
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.011792
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.042325
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.061765
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.091107
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.055689
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.102481
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.046042


Finally, we copy and paste the last 5 lines of the testing result:

Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.005480
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.103070
Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.003925
Train Epoch: 9 [58880/60000 (98%)]	Loss: 0.018122
Train Epoch: 9 [59520/60000 (99%)]	Loss: 0.025634

Test set: Average loss: 0.0484, Accuracy: 9849/10000 (98%)